In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
import seaborn as sns
sns.set_context('poster')

## Moving Average Models With MSA, Years and Features

### Import, Split, and Standardize Data

In [2]:
start = datetime.datetime.time(datetime.datetime.now())

In [3]:
df = pd.read_pickle('../data/merged/all_data_2006_to_2016_v2.pkl')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 928 entries, 0 to 927
Data columns (total 13 columns):
MSA_orig                                                                       928 non-null object
MSA                                                                            928 non-null object
MSA_abbr                                                                       928 non-null object
year                                                                           928 non-null int64
now_married_except_separated                                                   928 non-null float64
less_than_high_school_diploma                                                  928 non-null float64
unmarried_portion_of_women_15_to_50_years_who_had_a_birth_in_past_12_months    928 non-null float64
households_with_food_stamp_snap_benefits                                       928 non-null float64
percentage_married_couple_family                                               928 non-null float64


In [5]:
# drop extra MSA names
df = df.drop(['MSA_orig', 'MSA'], axis=1)

In [6]:
# remove outliers with atypically high murder rates
df = df[df.MSA_abbr != 'NEW_ORLEANS_LA']
df = df[(df.MSA_abbr != 'MEMPHIS_TN') | (df.year != 2016)]
df = df[(df.MSA_abbr != 'BATON_ROUGE_LA') | (df.year != 2007)]

In [7]:
df = df.sort_values(['MSA_abbr', 'year'], ascending=[True, True])

In [8]:
df_msa = df['MSA_abbr']

In [9]:
df = pd.get_dummies(df, columns=['MSA_abbr'], drop_first=False)

In [10]:
df['MSA_abbr'] = df_msa.values

In [11]:
cols = [df.columns[-1]] + [col for col in df if col != df.columns[-1]]
df = df[cols]

In [12]:
df.head()

,MSA_abbr,year,now_married_except_separated,less_than_high_school_diploma,unmarried_portion_of_women_15_to_50_years_who_had_a_birth_in_past_12_months,households_with_food_stamp_snap_benefits,percentage_married_couple_family,percentage_female_householder_no_husband_present_family,poverty_all_people,house_median_value_(dollars),...,MSA_abbr_TAMPA_FL,MSA_abbr_TOLEDO_OH,MSA_abbr_TUCSON_AZ,MSA_abbr_TULSA_OK,MSA_abbr_VIRGINIA_BEACH_NC,MSA_abbr_WASHINGTON_DC,MSA_abbr_WICHITA_KS,MSA_abbr_WINSTON_NC,MSA_abbr_WORCESTER_MA,MSA_abbr_YOUNGSTOWN_OH
0,AKRON_OH,2007,50.5,9.9,28.0,8.8,75.7,18.0,13.4,147000,...,0,0,0,0,0,0,0,0,0,0
1,AKRON_OH,2008,49.3,9.9,35.9,9.5,74.3,19.9,12.1,148300,...,0,0,0,0,0,0,0,0,0,0
2,AKRON_OH,2009,48.1,10.1,42.4,12.0,73.2,20.7,14.7,142500,...,0,0,0,0,0,0,0,0,0,0
3,AKRON_OH,2010,47.9,10.7,41.0,12.9,73.7,19.8,15.5,145000,...,0,0,0,0,0,0,0,0,0,0
4,AKRON_OH,2011,46.8,9.1,38.2,14.5,72.9,21.7,16.6,139800,...,0,0,0,0,0,0,0,0,0,0


In [13]:
list(df)[:15]

['MSA_abbr',
 'year',
 'now_married_except_separated',
 'less_than_high_school_diploma',
 'unmarried_portion_of_women_15_to_50_years_who_had_a_birth_in_past_12_months',
 'households_with_food_stamp_snap_benefits',
 'percentage_married_couple_family',
 'percentage_female_householder_no_husband_present_family',
 'poverty_all_people',
 'house_median_value_(dollars)',
 'murder_per_100_k',
 'MSA_abbr_AKRON_OH',
 'MSA_abbr_ALBANY_NY',
 'MSA_abbr_ALBUQUERQUE_NM',
 'MSA_abbr_ALLENTOWN_PA']

In [14]:
# train test split / separate labels and features

label_col = 'murder_per_100_k'

split_yr = 2011

df_train_s = df[df['year'] <= split_yr]
df_test = df[df['year'] > split_yr]

print('Len train: {}'.format(len(df_train_s)))
print('Len test: {}'.format(len(df_test)))

Len train: 490
Len test: 426


In [15]:
def moving_average(df, ma_yrs):
    min_yr = df.year.min()
    max_yr = df.year.max()
    cols = list(df)
    MSAs = df.MSA_abbr.unique()
    main_df = []
    for yr in range(min_yr, max_yr+1):
        window_min_yr = yr - int(ma_yrs/2)
        window_max_yr = yr + int(ma_yrs/2)
        for MSA in MSAs:
            MSA_row = np.array(df[(df.MSA_abbr == MSA) & (df.year == yr)])
            if MSA_row.size == 0:
                continue
            dummies = MSA_row[:, 11:]
            MSA_row = np.array(df[(df.MSA_abbr == MSA) & (df.year >= window_min_yr) & (df.year <= window_max_yr)])
            MSA_row = MSA_row[:, 2:11]
            MSA_row = np.mean(MSA_row, axis=0)
            MSA_row = np.append(yr, MSA_row)
            MSA_row = np.append(MSA, MSA_row)
            MSA_row = np.append(MSA_row, dummies)
            main_df.append(MSA_row)
    main_df = pd.DataFrame(main_df)
    main_df.columns = cols
    main_df = main_df.sort_values(['MSA_abbr', 'year'], ascending=[True, True])
    return main_df

In [16]:
df_train_s_ma = moving_average(df_train_s, 5)

In [17]:
df_train_s_ma.head()

,MSA_abbr,year,now_married_except_separated,less_than_high_school_diploma,unmarried_portion_of_women_15_to_50_years_who_had_a_birth_in_past_12_months,households_with_food_stamp_snap_benefits,percentage_married_couple_family,percentage_female_householder_no_husband_present_family,poverty_all_people,house_median_value_(dollars),...,MSA_abbr_TAMPA_FL,MSA_abbr_TOLEDO_OH,MSA_abbr_TUCSON_AZ,MSA_abbr_TULSA_OK,MSA_abbr_VIRGINIA_BEACH_NC,MSA_abbr_WASHINGTON_DC,MSA_abbr_WICHITA_KS,MSA_abbr_WINSTON_NC,MSA_abbr_WORCESTER_MA,MSA_abbr_YOUNGSTOWN_OH
44,AKRON_OH,2007,49.300,9.966667,35.433333,10.100000,74.400000,19.533333,13.400,145933.333333,...,0,0,0,0,0,0,0,0,0,0
128,AKRON_OH,2008,48.950,10.150000,36.825000,10.800000,74.225000,19.600000,13.925,145700.000000,...,0,0,0,0,0,0,0,0,0,0
217,AKRON_OH,2009,48.520,9.940000,37.100000,11.540000,73.960000,20.020000,14.460,144520.000000,...,0,0,0,0,0,0,0,0,0,0
304,AKRON_OH,2010,48.025,9.950000,39.375000,12.225000,73.525000,20.525000,14.725,143900.000000,...,0,0,0,0,0,0,0,0,0,0
397,AKRON_OH,2011,47.600,9.966667,40.533333,13.133333,73.266667,20.733333,15.600,142433.333333,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# train test split / separate labels and features

label_col = 'murder_per_100_k'

split_yr = 2014

x_train_s = df_train_s_ma.drop([label_col], axis=1).drop('MSA_abbr', axis=1)
x_test_s = df_test.drop([label_col], axis=1).drop('MSA_abbr', axis=1)
y_train = df_train_s_ma[label_col]
y_test = df_test[label_col]

print('Sizes match: {}'.format(len(x_train_s)==len(y_train)))
print()
print('Len x_train: {}'.format(len(x_train_s)))
print('Len x_test: {}'.format(len(x_test_s)))
print('Len y_train: {}'.format(len(y_train)))
print('Len x_test: {}'.format(len(y_test)))

Sizes match: True

Len x_train: 490
Len x_test: 426
Len y_train: 490
Len x_test: 426


In [19]:
# standardize data

from sklearn.preprocessing import StandardScaler
standardizer = StandardScaler().fit(x_train_s)

x_train = standardizer.transform(x_train_s)
x_test = standardizer.transform(x_test_s)

### Simple Linear Regression

In [20]:
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, BayesianRidge, HuberRegressor
from sklearn.model_selection import GridSearchCV

In [21]:
# instantiate and fit models

def make_models(x_train, y_train):
    md = dict()

    md['linear'] = LinearRegression().fit(x_train, y_train)
    md['ridge'] = RidgeCV(cv=15).fit(x_train, y_train)
    md['lasso'] = LassoCV(cv=15).fit(x_train, y_train)
    md['bayes'] = BayesianRidge(tol=0.0001).fit(x_train, y_train)
    md['huber'] = GridSearchCV(HuberRegressor(),{'epsilon': [1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7]}).fit(x_train, y_train).best_estimator_
    
    return md

In [22]:
# score models

def score_model(model):

    train_score = model.score(x_train, y_train)
    test_score = model.score(x_test, y_test)

    return np.array([train_score, test_score])

In [23]:
def results_df(model):

    # get train and test scores
    scores_df = pd.DataFrame(score_model(model)).transpose()
    scores_df.columns = ['Train R2','Test R2']

    # get coefficient matrix
    coeffs_df = pd.DataFrame(model.coef_).transpose()
    coeffs_df.columns = x_train_s.columns

    # join dataframes
    return pd.concat([scores_df, coeffs_df], axis=1)

In [24]:
from sklearn.utils import resample

In [25]:
def run_experiment(n_iters):
    
    sample_results = dict()
    
    for n in range(n_iters):
        # get new sample
        xb, yb = resample(x_train, y_train)

        # make and fit models
        model_dict = make_models(xb, yb)

        # get sample of results for each model
        for key in model_dict:
            
            # initialize empty dictionary
            if key not in sample_results:
                sample_results[key] = []
                        
            # get model results
            sample_results[key].append(results_df(model_dict[key]))
            
    # concatenate results dfs into single df
    for key in sample_results:
        sample_results[key] = pd.concat(sample_results[key])
        
    return sample_results

In [26]:
exp = run_experiment(100)

/Users/ilanjdor/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ilanjdor/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ilanjdor/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ilanjdor/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective di

/Users/ilanjdor/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ilanjdor/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ilanjdor/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ilanjdor/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective di

/Users/ilanjdor/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ilanjdor/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ilanjdor/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ilanjdor/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective di

/Users/ilanjdor/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [27]:
coef_dict = dict()

# iterate over all models
for key in exp:
    
    # iterate over results of this model
    for c in exp[key].columns:
        
        # initialize dict for result names
        if c not in coef_dict:
            coef_dict[c] = dict()
    
        # add this coeff to the dict
        coef_dict[c][key] = exp[key][c]

# convert dict of dicts into dict of dataframes
coef_dfs = {key: pd.DataFrame(coef_dict[key]) for key in coef_dict}

In [28]:
for key in exp:
    print(key)
    print()
    print(exp[key].mean()[:2])
    print()

linear

Train R2   -6.409481e+24
Test R2    -9.100402e+24
dtype: float64

ridge

Train R2    0.963465
Test R2     0.614645
dtype: float64

lasso

Train R2    0.963631
Test R2     0.582681
dtype: float64

bayes

Train R2    0.963921
Test R2     0.624212
dtype: float64

huber

Train R2    0.894911
Test R2     0.543365
dtype: float64



In [29]:
def print_runtime():
    hours = int(str(end)[0:2])-int(str(start)[0:2])
    minutes = int(str(end)[3:5])-int(str(start)[3:5])
    seconds = int(str(end)[6:8])-int(str(start)[6:8])
    if hours < 0:
        hours = hours + 24
    if minutes < 0:
        minutes = minutes + 60
        hours = hours - 1
    if seconds < 0:
        seconds = seconds + 60
        minutes = minutes - 1
    print(hours, "hrs", minutes, "mins", seconds, "secs")

In [30]:
end = datetime.datetime.time(datetime.datetime.now())

In [31]:
print_runtime()

0 hrs 7 mins 58 secs
